In [15]:

import json
import random
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.regularizers import l2
import gensim.downloader as api

In [16]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ajmel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [17]:
import nltk
nltk.download('punkt', 'C:/Users/ajmel/Desktop/try/Travel-Guidence-Chatbot/chatbot_backend/chatapi/utils/')
nltk.download('wordnet', 'C:/Users/ajmel/Desktop/try/Travel-Guidence-Chatbot/chatbot_backend/chatapi/utils/')

[nltk_data] Downloading package punkt to
[nltk_data]     C:/Users/ajmel/Desktop/try/Travel-Guidence-
[nltk_data]     Chatbot/chatbot_backend/chatapi/utils/...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:/Users/ajmel/Desktop/try/Travel-Guidence-
[nltk_data]     Chatbot/chatbot_backend/chatapi/utils/...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
import nltk
print(nltk.data.path)

['C:\\Users\\ajmel/nltk_data', 'c:\\Users\\ajmel\\Desktop\\try\\Travel-Guidence-Chatbot\\myenv\\nltk_data', 'c:\\Users\\ajmel\\Desktop\\try\\Travel-Guidence-Chatbot\\myenv\\share\\nltk_data', 'c:\\Users\\ajmel\\Desktop\\try\\Travel-Guidence-Chatbot\\myenv\\lib\\nltk_data', 'C:\\Users\\ajmel\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [19]:
# Load and parse JSON data
with open('C:/Users/ajmel/Desktop/try/Travel-Guidence-Chatbot/chatbot_backend/chatapi/utils/baale_mountain.json', encoding='utf-8') as file:
    intents = json.load(file)

lemmatizer = WordNetLemmatizer()

# Process patterns and tags
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ","] 
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize and lemmatize
        tokens = nltk.word_tokenize(pattern)
        words.extend([lemmatizer.lemmatize(word.lower()) for word in tokens])
        documents.append((tokens, intent['tag']))
    
    # Collect unique classes
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Remove duplicates and sort
words = [lemmatizer.lemmatize(word) 
         for word in words if word not in ignore_letters] 
words = sorted(set(words))
classes = sorted(set(classes))

# Save vocabulary and classes
with open('vocabulary.pkl', 'wb') as f:
    pickle.dump((words, classes), f)

In [22]:
# Create bag-of-words training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    
    # Create BoW vector
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # Create output vector
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [23]:
# Shuffle and split data
random.shuffle(training)
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

# Calculate class weights (if class imbalance exists)
class_weights = compute_class_weight('balanced', classes=np.unique(train_y.argmax(axis=1)), y=train_y.argmax(axis=1))
class_weight_dict = {i: class_weights[i] for i in range(len(classes))}

# Load pre-trained embeddings (GloVe)
glove_vectors = api.load("glove-wiki-gigaword-100")  # You can change to different word vectors if needed

# Convert the words into GloVe embeddings
embedding_dim = 100
embedding_matrix = np.zeros((len(words), embedding_dim))
for i, word in enumerate(words):
    if word in glove_vectors:
        embedding_matrix[i] = glove_vectors[word]

[==================================================] 100.0% 128.1/128.1MB downloaded


In [24]:
from tensorflow.keras.callbacks import EarlyStopping

# Build the model
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
#model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
#model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation='softmax'))

adam = Adam(learning_rate=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', 
              optimizer=adam, 
              metrics=['accuracy'])
# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.summary()

c:\Users\ajmel\Desktop\try\Travel-Guidence-Chatbot\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ajmel\Desktop\try\Travel-Guidence-Chatbot\myenv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        44,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,024 (312.59 KB)

 Trainable params: 80,024 (312.59 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Implementing Stratified K-Fold cross-validation for improved generalization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
for train_idx, val_idx in kfold.split(train_x, train_y.argmax(axis=1)):
    print(f"\nTraining fold {fold}...")
    # Split into training and validation sets for this fold
    X_train, X_val = train_x[train_idx], train_x[val_idx]
    y_train, y_val = train_y[train_idx], train_y[val_idx]
    
    # Train model on the current fold
    history = model.fit(X_train, y_train, 
                        epochs=200, 
                        batch_size=5, 
                        verbose=1,
                        validation_data=(X_val, y_val), 
                        class_weight=class_weight_dict)  # Add class weight to handle class imbalance

    fold += 1

# Save model and metadata
model.save('C:/Users/ajmel/Desktop/try/Travel-Guidence-Chatbot/chatbot_backend/chatapi/utils/chatbot_model.keras')


Training fold 1...
Epoch 1/200
22/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9404 - loss: 0.4957 

c:\Users\ajmel\Desktop\try\Travel-Guidence-Chatbot\myenv\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9309 - loss: 0.5218 - val_accuracy: 1.0000 - val_loss: 0.3959
Epoch 2/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9701 - loss: 0.5047 - val_accuracy: 1.0000 - val_loss: 0.4271
Epoch 3/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9712 - loss: 0.5773 - val_accuracy: 0.9688 - val_loss: 0.4536
Epoch 4/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9935 - loss: 0.4459 - val_accuracy: 1.0000 - val_loss: 0.4456
Epoch 5/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9866 - loss: 0.4578 - val_accuracy: 1.0000 - val_loss: 0.4185
Epoch 6/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.3981 - val_accuracy: 0.9688 - val_loss: 0.4123
Epoch 7/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9962 - loss: 0.3850 - val_accuracy: 0.9688 - val_loss: 0.4063
Epoch 8/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9982 - loss: 0.3630 - val_accuracy: 0.9062 - val_loss: 0.4

In [27]:
# After processing patterns
print("Final Vocabulary Size:", len(words))  
print("Sample Vocabulary:", words[:10])
print("Final classe Size:", len(classes))
print("Sample Classes:", classes)

# Before saving
with open('C:/Users/ajmel/Desktop/try/Travel-Guidence-Chatbot/chatbot_backend/chatapi/utils/vocabulary.pkl', 'wb') as f:
    pickle.dump(words, f)  # NOT lemmatized patterns

Final Vocabulary Size: 171
Sample Vocabulary: ["'s", '3-day', 'a', 'about', 'accommodation', 'activity', 'afroalpine', 'afternoon', 'and', 'animal']
Final classe Size: 24
Sample Classes: ['Get Northern Grasslands (Gaysay Grasslands) Information', 'GetAfroalpineMeadowsInformation', 'GetEricaBeltInformation', 'GetHarennaForestInformation', 'GetJuniperWoodlandsInformation', 'GetParkParts', 'activities_within_park', 'birds', 'bmn_service_provider', 'farewell', 'greeting', 'health_and_safety', 'lodging', 'nearby_attractions', 'park_fees', 'place_History', 'place_info', 'time_based_greeting', 'travel_directory', 'trip_itineraries', 'weather', 'what_to_bring', 'when_to_go', 'wildlife']


In [28]:
# After training data generation
import pickle

with open('classes.pkl', 'wb') as f:
    pickle.dump(classes, f)  # Add this line
    
with open('vocabulary.pkl', 'wb') as f:
    pickle.dump(words, f)

In [29]:
from sklearn.metrics import accuracy_score, classification_report, f1_score

# Predict class probabilities for validation set
y_pred_probs = model.predict(X_val)

# Convert probabilities to class labels (one-hot to single label)
y_val_labels = np.argmax(y_val, axis=1)
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_val_labels, y_pred_labels)
f1 = f1_score(y_val_labels, y_pred_labels, average='weighted')  # weighted for class imbalance
report = classification_report(y_val_labels, y_pred_labels, target_names=classes, labels=np.unique(y_val_labels))

# Display results
print(f"\n✅ Validation Accuracy: {accuracy:.4f}")
print(f"✅ Validation F1 Score: {f1:.4f}")
print("\nClassification Report:\n", report)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

✅ Validation Accuracy: 0.8710
✅ Validation F1 Score: 0.8430

Classification Report:
                                                          precision    recall  f1-score   support

Get Northern Grasslands (Gaysay Grasslands) Information       1.00      1.00      1.00         1
                        GetAfroalpineMeadowsInformation       1.00      1.00      1.00         1
                                GetEricaBeltInformation       1.00      1.00      1.00         1
                            GetHarennaForestInformation       1.00      1.00      1.00         1
                         GetJuniperWoodlandsInformation       0.00      0.00      0.00         1
                                           GetParkParts       1.00      1.00      1.00         2
                                 activities_within_park       0.67      1.00      0.80         2
                                                  birds       0.67      1.00      0.80         2
  

c:\Users\ajmel\Desktop\try\Travel-Guidence-Chatbot\myenv\Lib\site-packages\sklearn\metrics\_classification.py:2687: UserWarning: labels size, 19, does not match size of target_names, 24
  warnings.warn(
c:\Users\ajmel\Desktop\try\Travel-Guidence-Chatbot\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ajmel\Desktop\try\Travel-Guidence-Chatbot\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ajmel\Desktop\try\Travel-Guidence-Chatbot\myenv\Lib\site-packages\sklearn\metrics\_c

In [30]:
import json

# Load your JSON file
with open('C:/Users/ajmel/Desktop/try/Travel-Guidence-Chatbot/chatbot_backend/chatapi/utils/baale_mountain.json', encoding='utf-8') as file:
    data = json.load(file)

# Check how many intents
num_intents = len(data['intents'])

# Check the number of patterns in each intent
for intent in data['intents']:
    num_patterns = len(intent['patterns'])
    print(f"Intent: {intent['tag']}, Number of patterns: {num_patterns}")

# Print the number of intents
print(f"Total number of intents: {num_intents}")


Intent: greeting, Number of patterns: 8
Intent: time_based_greeting, Number of patterns: 3
Intent: place_info, Number of patterns: 7
Intent: place_History, Number of patterns: 8
Intent: trip_itineraries, Number of patterns: 3
Intent: activities_within_park, Number of patterns: 10
Intent: nearby_attractions, Number of patterns: 10
Intent: lodging, Number of patterns: 3
Intent: when_to_go, Number of patterns: 13
Intent: what_to_bring, Number of patterns: 10
Intent: health_and_safety, Number of patterns: 3
Intent: park_fees, Number of patterns: 11
Intent: bmn_service_provider, Number of patterns: 11
Intent: travel_directory, Number of patterns: 10
Intent: weather, Number of patterns: 4
Intent: GetParkParts, Number of patterns: 5
Intent: GetHarennaForestInformation, Number of patterns: 4
Intent: Get Northern Grasslands (Gaysay Grasslands) Information, Number of patterns: 4
Intent: GetJuniperWoodlandsInformation, Number of patterns: 4
Intent: GetAfroalpineMeadowsInformation, Number of patte